# S5E11 Loan Payback - V13: Seed Ensemble + Pseudo-labeling

## Strategy
1. **V12 피처 엔지니어링** (Enhanced Interactions)
2. **Seed Ensemble**: 4개 시드로 분산 줄이기
3. **Pseudo-labeling**: 확신도 높은 예측 활용
4. **XGBoost + CatBoost** 앙상블

### Target: 0.928+ Public LB

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

try:
    from cuml.preprocessing import TargetEncoder
    USE_CUML_TE = True
    print("Using cuml.preprocessing.TargetEncoder")
except ImportError:
    USE_CUML_TE = False

print("="*80)
print("V13: Seed Ensemble + Pseudo-labeling")
print("="*80)

# Seeds for ensemble
SEEDS = [42, 123, 456, 789]
print(f"\nSeeds: {SEEDS}")

train = pd.read_csv('/kaggle/input/playground-series-s5e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e11/test.csv')
orig = pd.read_csv('/kaggle/input/loan-prediction-dataset-2025/loan_dataset_20000.csv')

print(f"\nTrain: {train.shape}, Test: {test.shape}, Orig: {orig.shape}")

TARGET = 'loan_paid_back'
CATS_BASE = ['gender', 'marital_status', 'education_level', 'employment_status', 
             'loan_purpose', 'grade_subgrade']
NUMS_BASE = ['annual_income', 'debt_to_income_ratio', 'credit_score', 
             'loan_amount', 'interest_rate']

test[TARGET] = -1
combine = pd.concat([train, test, orig], axis=0, ignore_index=True)

## Feature Engineering (V12와 동일)

In [ ]:
print("\n[Feature Engineering]...")

# Financial features
combine['income_loan_ratio'] = combine['annual_income'] / (combine['loan_amount'] + 1)
combine['loan_to_income'] = combine['loan_amount'] / (combine['annual_income'] + 1)
combine['total_debt'] = combine['debt_to_income_ratio'] * combine['annual_income']
combine['available_income'] = combine['annual_income'] * (1 - combine['debt_to_income_ratio'])
combine['debt_burden'] = combine['debt_to_income_ratio'] * combine['loan_amount']
combine['monthly_payment'] = combine['loan_amount'] * combine['interest_rate'] / 1200
combine['payment_to_income'] = combine['monthly_payment'] / (combine['annual_income'] / 12 + 1)
combine['affordability'] = combine['available_income'] / (combine['loan_amount'] + 1)
combine['default_risk'] = (combine['debt_to_income_ratio'] * 0.40 + 
                          (850 - combine['credit_score']) / 850 * 0.35 + 
                          combine['interest_rate'] / 100 * 0.25)
combine['credit_utilization'] = combine['credit_score'] * (1 - combine['debt_to_income_ratio'])
combine['credit_interest_product'] = combine['credit_score'] * combine['interest_rate'] / 100
combine['income_debt_ratio'] = combine['annual_income'] / (combine['total_debt'] + 1)
combine['credit_debt_ratio'] = combine['credit_score'] / (combine['debt_to_income_ratio'] + 0.01)
combine['risk_adjusted_income'] = combine['annual_income'] * (1 - combine['default_risk'])

for col in ['annual_income', 'loan_amount', 'total_debt']:
    combine[f'{col}_log'] = np.log1p(combine[col])

combine['grade_letter'] = combine['grade_subgrade'].str[0]
combine['grade_number'] = combine['grade_subgrade'].str[1].astype(int)
grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
combine['grade_rank'] = combine['grade_letter'].map(grade_map)
combine['grade_combined'] = combine['grade_rank'] * 10 + combine['grade_number']

# ROUND features
ROUND_FEATURES = []
for col in ['annual_income', 'loan_amount']:
    for suffix, level in {'1000s': -3, '100s': -2, '10s': -1, '1s': 0}.items():
        new_col = f'{col}_ROUND_{suffix}'
        ROUND_FEATURES.append(new_col)
        combine[new_col] = combine[col].round(level).astype(int)

# Credit tiers
def map_fico(s): 
    if s >= 800: return 'Exceptional'
    elif s >= 740: return 'Very Good'
    elif s >= 670: return 'Good'
    elif s >= 580: return 'Fair'
    else: return 'Poor'

def map_vantage(s):
    if s >= 781: return 'Excellent'
    elif s >= 661: return 'Good'
    elif s >= 601: return 'Fair'
    elif s >= 500: return 'Poor'
    else: return 'Very Poor'

combine['credit_score_FICO_tier'] = combine['credit_score'].apply(map_fico)
combine['credit_score_Vantage_tier'] = combine['credit_score'].apply(map_vantage)

TIER_FEATURES = ['credit_score_FICO_tier', 'credit_score_Vantage_tier']
print("Financial, ROUND, Tier features created!")

In [ ]:
# Define feature lists
V12_NUM_FEATURES = ['income_loan_ratio', 'loan_to_income', 'total_debt', 
                    'available_income', 'debt_burden', 'monthly_payment',
                    'payment_to_income', 'affordability', 'default_risk',
                    'credit_utilization', 'credit_interest_product',
                    'income_debt_ratio', 'credit_debt_ratio', 'risk_adjusted_income',
                    'annual_income_log', 'loan_amount_log', 'total_debt_log',
                    'grade_number', 'grade_rank', 'grade_combined']

CATS = CATS_BASE + ['grade_letter'] + TIER_FEATURES
NUMS = NUMS_BASE + V12_NUM_FEATURES + ROUND_FEATURES

# Factorize
CATS_NUM = []
for c in NUMS:
    n = f"{c}_cat"
    CATS_NUM.append(n)
    combine[n], _ = combine[c].factorize()
    combine[n] = combine[n].astype('int32')

print(f"CATS: {len(CATS)}, NUMS: {len(NUMS)}, CATS_NUM: {len(CATS_NUM)}")

In [ ]:
# Interactions
v10_pairs = [
    ('employment_status', 'grade_subgrade'), ('employment_status', 'education_level'),
    ('employment_status', 'loan_purpose'), ('grade_subgrade', 'loan_purpose'),
    ('grade_subgrade', 'education_level'), ('marital_status', 'employment_status'),
    ('credit_score_FICO_tier', 'employment_status'), ('credit_score_FICO_tier', 'grade_subgrade'),
    ('credit_score_FICO_tier', 'loan_purpose'), ('credit_score_Vantage_tier', 'employment_status'),
    ('credit_score_Vantage_tier', 'grade_subgrade'), ('grade_letter', 'employment_status'),
    ('grade_letter', 'education_level'), ('grade_letter', 'loan_purpose'),
    ('gender', 'employment_status'), ('gender', 'education_level'),
    ('gender', 'marital_status'), ('marital_status', 'education_level'),
    ('marital_status', 'loan_purpose'),
]

for num_cat in ['credit_score_cat', 'debt_to_income_ratio_cat', 'interest_rate_cat', 
                'default_risk_cat', 'affordability_cat', 'payment_to_income_cat']:
    for cat in ['employment_status', 'grade_subgrade', 'loan_purpose']:
        v10_pairs.append((num_cat, cat))

for round_cat in ['annual_income_ROUND_1000s_cat', 'loan_amount_ROUND_1000s_cat']:
    for cat in ['grade_subgrade', 'employment_status', 'loan_purpose', 'education_level']:
        v10_pairs.append((round_cat, cat))

three_way = [
    ('employment_status', 'grade_letter', 'credit_score_FICO_tier'),
    ('employment_status', 'loan_purpose', 'grade_letter'),
    ('education_level', 'employment_status', 'grade_letter'),
]

CATS_INTER = []
for c1, c2 in v10_pairs:
    name = f"{c1}_{c2}"
    if c1 in combine.columns and c2 in combine.columns and name not in combine.columns:
        combine[name] = combine[c1].astype(str) + '_' + combine[c2].astype(str)
        CATS_INTER.append(name)

for c1, c2, c3 in three_way:
    name = f"{c1}_{c2}_{c3}"
    if name not in combine.columns:
        combine[name] = combine[c1].astype(str) + '_' + combine[c2].astype(str) + '_' + combine[c3].astype(str)
        CATS_INTER.append(name)

print(f"Interactions: {len(CATS_INTER)}")

In [ ]:
# Count Encoding
CE = []
ALL_CATS = CATS + CATS_NUM + CATS_INTER
for c in ALL_CATS:
    tmp = combine.groupby(c)[TARGET].count()
    tmp.name = f"CE_{c}"
    CE.append(f"CE_{c}")
    combine = combine.merge(tmp, on=c, how='left')

print(f"Count Encoding: {len(CE)}")

In [ ]:
# Original data features
train_len, test_len, orig_len = len(train), len(test), len(orig)

train_tmp = combine.iloc[:train_len].copy()
test_tmp = combine.iloc[train_len:train_len + test_len].copy()
orig_tmp = combine.iloc[-orig_len:].copy()

ORIG_FEATURES = []
for col in CATS_BASE + ['grade_letter'] + TIER_FEATURES:
    mean_map = orig_tmp.groupby(col)[TARGET].mean()
    mean_col = f"orig_mean_{col}"
    mean_map.name = mean_col
    train_tmp = train_tmp.merge(mean_map, on=col, how='left')
    test_tmp = test_tmp.merge(mean_map, on=col, how='left')
    orig_tmp = orig_tmp.merge(mean_map, on=col, how='left')
    ORIG_FEATURES.append(mean_col)
    
    count_map = orig_tmp.groupby(col).size().reset_index(name=f"orig_count_{col}")
    train_tmp = train_tmp.merge(count_map, on=col, how='left')
    test_tmp = test_tmp.merge(count_map, on=col, how='left')
    orig_tmp = orig_tmp.merge(count_map, on=col, how='left')
    ORIG_FEATURES.append(f"orig_count_{col}")

combine = pd.concat([train_tmp, test_tmp, orig_tmp], axis=0, ignore_index=True)
print(f"Original features: {len(ORIG_FEATURES)}")

In [ ]:
# Final split
train = combine.iloc[:train_len].copy()
test = combine.iloc[train_len:train_len + test_len].copy()
orig = combine.iloc[-orig_len:].copy()

FEATURES = NUMS + CATS + CATS_NUM + CATS_INTER + CE + ORIG_FEATURES
TARGET_ENCODE_CATS = CATS_NUM + CATS_INTER

print(f"\nTOTAL FEATURES: {len(FEATURES)}")

## Model Configuration

In [ ]:
FOLDS = 8

def get_xgb_params(seed):
    return {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "gpu_hist",
        "device": "cuda",
        "learning_rate": 0.01,
        "max_depth": 0,
        "subsample": 0.8,
        "colsample_bytree": 0.7,
        "seed": seed,
        "grow_policy": "lossguide",
        "max_leaves": 128,
        "scale_pos_weight": 0.78,
        "reg_lambda": 5.0,
        "reg_alpha": 2.5,
        "max_bin": 256,
        "verbosity": 0,
    }

def get_cat_params(seed):
    return {
        'iterations': 15000,
        'learning_rate': 0.02,
        'depth': 6,
        'l2_leaf_reg': 10,
        'border_count': 254,
        'eval_metric': 'AUC',
        'early_stopping_rounds': 300,
        'verbose': 500,
        'random_seed': seed,
        'use_best_model': True,
        'task_type': 'GPU',
    }

print(f"Seeds: {SEEDS}")
print(f"Folds: {FOLDS}")
print(f"Total models per seed: {FOLDS * 2} (XGB + CAT)")
print(f"Total models: {len(SEEDS) * FOLDS * 2}")

## Seed Ensemble Training

In [ ]:
print(f"\n{'='*80}")
print("SEED ENSEMBLE TRAINING")
print(f"{'='*80}")

# Store predictions for each seed
all_xgb_oof = []
all_xgb_test = []
all_cat_oof = []
all_cat_test = []
all_seed_scores = []

for seed_idx, SEED in enumerate(SEEDS):
    print(f"\n{'#'*80}")
    print(f"### SEED {SEED} ({seed_idx+1}/{len(SEEDS)}) ###")
    print(f"{'#'*80}")
    
    xgb_params = get_xgb_params(SEED)
    catboost_params = get_cat_params(SEED)
    
    xgb_oof = np.zeros(len(train))
    xgb_test = np.zeros(len(test))
    cat_oof = np.zeros(len(train))
    cat_test = np.zeros(len(test))
    
    kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
        print(f"\n  Seed {SEED} - Fold {fold+1}/{FOLDS}")
        
        Xy_train = train.iloc[train_idx][FEATURES + [TARGET]].copy()
        Xy_orig = orig[FEATURES + [TARGET]].copy()
        Xy_train = pd.concat([Xy_train, Xy_orig], axis=0, ignore_index=True)
        
        X_valid = train.iloc[val_idx][FEATURES].copy()
        y_valid = train.iloc[val_idx][TARGET]
        X_test_fold = test[FEATURES].copy()
        
        # Target Encoding
        if USE_CUML_TE:
            Xy_train_te = Xy_train.copy()
            X_valid_te = X_valid.copy()
            X_test_te = X_test_fold.copy()
            
            for col in TARGET_ENCODE_CATS:
                te = TargetEncoder(n_folds=10, smooth=1.0, split_method='random', stat='mean')
                Xy_train_te[col] = te.fit_transform(Xy_train[[col]], Xy_train[TARGET]).astype('float32')
                X_valid_te[col] = te.transform(X_valid[[col]]).astype('float32')
                X_test_te[col] = te.transform(X_test_fold[[col]]).astype('float32')
        
        X_train_final = Xy_train_te[FEATURES].copy()
        y_train_final = Xy_train_te[TARGET]
        X_valid_final = X_valid_te[FEATURES].copy()
        X_test_final = X_test_te[FEATURES].copy()
        
        # XGBoost
        X_train_xgb = X_train_final.copy()
        X_valid_xgb = X_valid_final.copy()
        X_test_xgb = X_test_final.copy()
        for c in CATS:
            X_train_xgb[c] = X_train_xgb[c].astype('category')
            X_valid_xgb[c] = X_valid_xgb[c].astype('category')
            X_test_xgb[c] = X_test_xgb[c].astype('category')
        
        dtrain = xgb.DMatrix(X_train_xgb, label=y_train_final, enable_categorical=True)
        dval = xgb.DMatrix(X_valid_xgb, label=y_valid, enable_categorical=True)
        dtest = xgb.DMatrix(X_test_xgb, enable_categorical=True)
        
        xgb_model = xgb.train(
            params=xgb_params, dtrain=dtrain, num_boost_round=10000,
            evals=[(dval, "valid")], early_stopping_rounds=300, verbose_eval=False
        )
        
        xgb_oof[val_idx] = xgb_model.predict(dval, iteration_range=(0, xgb_model.best_iteration + 1))
        xgb_test += xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration + 1)) / FOLDS
        
        # CatBoost
        X_train_cat = X_train_final.copy()
        X_valid_cat = X_valid_final.copy()
        X_test_cat = X_test_final.copy()
        for c in CATS:
            X_train_cat[c] = X_train_cat[c].astype(str)
            X_valid_cat[c] = X_valid_cat[c].astype(str)
            X_test_cat[c] = X_test_cat[c].astype(str)
        
        cat_model = CatBoostClassifier(**catboost_params)
        cat_model.fit(X_train_cat, y_train_final, eval_set=(X_valid_cat, y_valid), 
                      cat_features=CATS, verbose=False)
        
        cat_oof[val_idx] = cat_model.predict_proba(X_valid_cat)[:, 1]
        cat_test += cat_model.predict_proba(X_test_cat)[:, 1] / FOLDS
        
        xgb_auc = roc_auc_score(y_valid, xgb_oof[val_idx])
        cat_auc = roc_auc_score(y_valid, cat_oof[val_idx])
        print(f"    XGB: {xgb_auc:.5f}, CAT: {cat_auc:.5f}")
    
    # Store seed results
    all_xgb_oof.append(xgb_oof)
    all_xgb_test.append(xgb_test)
    all_cat_oof.append(cat_oof)
    all_cat_test.append(cat_test)
    
    xgb_seed_auc = roc_auc_score(train[TARGET], xgb_oof)
    cat_seed_auc = roc_auc_score(train[TARGET], cat_oof)
    blend_oof = 0.4 * xgb_oof + 0.6 * cat_oof
    blend_seed_auc = roc_auc_score(train[TARGET], blend_oof)
    
    all_seed_scores.append((SEED, xgb_seed_auc, cat_seed_auc, blend_seed_auc))
    print(f"\n  Seed {SEED} Results: XGB={xgb_seed_auc:.5f}, CAT={cat_seed_auc:.5f}, Blend={blend_seed_auc:.5f}")

## Seed Ensemble Results

In [ ]:
print("\n" + "="*80)
print("SEED ENSEMBLE RESULTS")
print("="*80)

# Average across seeds
xgb_oof_ensemble = np.mean(all_xgb_oof, axis=0)
xgb_test_ensemble = np.mean(all_xgb_test, axis=0)
cat_oof_ensemble = np.mean(all_cat_oof, axis=0)
cat_test_ensemble = np.mean(all_cat_test, axis=0)

xgb_ensemble_auc = roc_auc_score(train[TARGET], xgb_oof_ensemble)
cat_ensemble_auc = roc_auc_score(train[TARGET], cat_oof_ensemble)

print(f"\nPer-Seed Results:")
print(f"{'Seed':<8} {'XGBoost':<12} {'CatBoost':<12} {'Blend':<12}")
print("-" * 44)
for seed, xgb_auc, cat_auc, blend_auc in all_seed_scores:
    print(f"{seed:<8} {xgb_auc:<12.5f} {cat_auc:<12.5f} {blend_auc:<12.5f}")

print(f"\nSeed Ensemble (Average of {len(SEEDS)} seeds):")
print(f"  XGBoost:  {xgb_ensemble_auc:.5f}")
print(f"  CatBoost: {cat_ensemble_auc:.5f}")

# Find optimal blend weight
best_w, best_blend_auc = 0.4, 0
for w in np.arange(0.0, 1.01, 0.05):
    blend = w * xgb_oof_ensemble + (1-w) * cat_oof_ensemble
    auc = roc_auc_score(train[TARGET], blend)
    if auc > best_blend_auc:
        best_blend_auc = auc
        best_w = w

print(f"  Blend:    {best_blend_auc:.5f} (XGB:{best_w:.2f} + CAT:{1-best_w:.2f})")

# Calculate variance reduction
single_seed_std = np.std([s[3] for s in all_seed_scores])
print(f"\nSingle seed std: {single_seed_std:.5f}")
print(f"Variance reduced by seed ensemble!")

## Pseudo-labeling (Round 2)

In [ ]:
print("\n" + "="*80)
print("PSEUDO-LABELING (ROUND 2)")
print("="*80)

# Use ensemble predictions for pseudo-labels
blend_test_r1 = best_w * xgb_test_ensemble + (1-best_w) * cat_test_ensemble

CONF_HIGH = 0.95
CONF_LOW = 0.05

pseudo_mask = (blend_test_r1 >= CONF_HIGH) | (blend_test_r1 <= CONF_LOW)
test_pseudo = test[pseudo_mask].copy()
test_pseudo[TARGET] = (blend_test_r1[pseudo_mask] >= 0.5).astype(float)

print(f"\nPseudo-labels: {len(test_pseudo)} ({pseudo_mask.mean()*100:.1f}%)")
print(f"  Class 0: {(test_pseudo[TARGET] == 0).sum()}")
print(f"  Class 1: {(test_pseudo[TARGET] == 1).sum()}")

In [ ]:
print("\n[Round 2 Training with Pseudo-labels + Seed Ensemble]")

all_xgb_oof_r2 = []
all_xgb_test_r2 = []
all_cat_oof_r2 = []
all_cat_test_r2 = []

for seed_idx, SEED in enumerate(SEEDS):
    print(f"\n  Seed {SEED} ({seed_idx+1}/{len(SEEDS)})...")
    
    xgb_params = get_xgb_params(SEED)
    catboost_params = get_cat_params(SEED)
    
    xgb_oof = np.zeros(len(train))
    xgb_test = np.zeros(len(test))
    cat_oof = np.zeros(len(train))
    cat_test = np.zeros(len(test))
    
    kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
        Xy_train = train.iloc[train_idx][FEATURES + [TARGET]].copy()
        Xy_orig = orig[FEATURES + [TARGET]].copy()
        Xy_pseudo = test_pseudo[FEATURES + [TARGET]].copy()
        Xy_train = pd.concat([Xy_train, Xy_orig, Xy_pseudo], axis=0, ignore_index=True)
        
        X_valid = train.iloc[val_idx][FEATURES].copy()
        y_valid = train.iloc[val_idx][TARGET]
        X_test_fold = test[FEATURES].copy()
        
        if USE_CUML_TE:
            Xy_train_te = Xy_train.copy()
            X_valid_te = X_valid.copy()
            X_test_te = X_test_fold.copy()
            for col in TARGET_ENCODE_CATS:
                te = TargetEncoder(n_folds=10, smooth=1.0, split_method='random', stat='mean')
                Xy_train_te[col] = te.fit_transform(Xy_train[[col]], Xy_train[TARGET]).astype('float32')
                X_valid_te[col] = te.transform(X_valid[[col]]).astype('float32')
                X_test_te[col] = te.transform(X_test_fold[[col]]).astype('float32')
        
        X_train_final = Xy_train_te[FEATURES].copy()
        y_train_final = Xy_train_te[TARGET]
        X_valid_final = X_valid_te[FEATURES].copy()
        X_test_final = X_test_te[FEATURES].copy()
        
        # XGBoost
        X_train_xgb = X_train_final.copy()
        X_valid_xgb = X_valid_final.copy()
        X_test_xgb = X_test_final.copy()
        for c in CATS:
            X_train_xgb[c] = X_train_xgb[c].astype('category')
            X_valid_xgb[c] = X_valid_xgb[c].astype('category')
            X_test_xgb[c] = X_test_xgb[c].astype('category')
        
        dtrain = xgb.DMatrix(X_train_xgb, label=y_train_final, enable_categorical=True)
        dval = xgb.DMatrix(X_valid_xgb, label=y_valid, enable_categorical=True)
        dtest = xgb.DMatrix(X_test_xgb, enable_categorical=True)
        
        xgb_model = xgb.train(
            params=xgb_params, dtrain=dtrain, num_boost_round=10000,
            evals=[(dval, "valid")], early_stopping_rounds=300, verbose_eval=False
        )
        xgb_oof[val_idx] = xgb_model.predict(dval, iteration_range=(0, xgb_model.best_iteration + 1))
        xgb_test += xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration + 1)) / FOLDS
        
        # CatBoost
        X_train_cat = X_train_final.copy()
        X_valid_cat = X_valid_final.copy()
        X_test_cat = X_test_final.copy()
        for c in CATS:
            X_train_cat[c] = X_train_cat[c].astype(str)
            X_valid_cat[c] = X_valid_cat[c].astype(str)
            X_test_cat[c] = X_test_cat[c].astype(str)
        
        cat_model = CatBoostClassifier(**catboost_params)
        cat_model.fit(X_train_cat, y_train_final, eval_set=(X_valid_cat, y_valid), 
                      cat_features=CATS, verbose=False)
        cat_oof[val_idx] = cat_model.predict_proba(X_valid_cat)[:, 1]
        cat_test += cat_model.predict_proba(X_test_cat)[:, 1] / FOLDS
    
    all_xgb_oof_r2.append(xgb_oof)
    all_xgb_test_r2.append(xgb_test)
    all_cat_oof_r2.append(cat_oof)
    all_cat_test_r2.append(cat_test)
    
    blend_oof = 0.4 * xgb_oof + 0.6 * cat_oof
    print(f"    Blend OOF: {roc_auc_score(train[TARGET], blend_oof):.5f}")

In [ ]:
print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

# Round 2 ensemble
xgb_oof_r2_ensemble = np.mean(all_xgb_oof_r2, axis=0)
xgb_test_r2_ensemble = np.mean(all_xgb_test_r2, axis=0)
cat_oof_r2_ensemble = np.mean(all_cat_oof_r2, axis=0)
cat_test_r2_ensemble = np.mean(all_cat_test_r2, axis=0)

xgb_r2_auc = roc_auc_score(train[TARGET], xgb_oof_r2_ensemble)
cat_r2_auc = roc_auc_score(train[TARGET], cat_oof_r2_ensemble)

# Find optimal blend
best_w_r2, best_blend_r2 = 0.4, 0
for w in np.arange(0.0, 1.01, 0.05):
    blend = w * xgb_oof_r2_ensemble + (1-w) * cat_oof_r2_ensemble
    auc = roc_auc_score(train[TARGET], blend)
    if auc > best_blend_r2:
        best_blend_r2 = auc
        best_w_r2 = w

print(f"\nRound 1 (Seed Ensemble):")
print(f"  XGBoost:  {xgb_ensemble_auc:.5f}")
print(f"  CatBoost: {cat_ensemble_auc:.5f}")
print(f"  Blend:    {best_blend_auc:.5f}")

print(f"\nRound 2 (+ Pseudo-labeling):")
print(f"  XGBoost:  {xgb_r2_auc:.5f}")
print(f"  CatBoost: {cat_r2_auc:.5f}")
print(f"  Blend:    {best_blend_r2:.5f}")

print(f"\nImprovement: {best_blend_r2 - best_blend_auc:+.5f}")
print(f"\nTarget: 0.928")
print(f"Gap:    {0.928 - best_blend_r2:.5f}")

In [ ]:
print("\n[Creating Submissions]")

# Final predictions (Blend)
final_test = best_w_r2 * xgb_test_r2_ensemble + (1-best_w_r2) * cat_test_r2_ensemble

submission = pd.DataFrame({'id': test['id'], TARGET: final_test})
submission.to_csv('submission.csv', index=False)
print(f"Saved: submission.csv (Round 2 Seed Ensemble Blend)")

# Round 1 for comparison
r1_test = best_w * xgb_test_ensemble + (1-best_w) * cat_test_ensemble
pd.DataFrame({'id': test['id'], TARGET: r1_test}).to_csv('submission_r1.csv', index=False)
print(f"Saved: submission_r1.csv (Round 1 Seed Ensemble Blend)")

# CatBoost only submissions
pd.DataFrame({'id': test['id'], TARGET: cat_test_r2_ensemble}).to_csv('submission_catboost_r2.csv', index=False)
print(f"Saved: submission_catboost_r2.csv (CatBoost Only, Round 2)")

pd.DataFrame({'id': test['id'], TARGET: cat_test_ensemble}).to_csv('submission_catboost_r1.csv', index=False)
print(f"Saved: submission_catboost_r1.csv (CatBoost Only, Round 1)")

In [ ]:
print("\n" + "="*80)
print("V13 SUMMARY")
print("="*80)

print(f"\nSeeds: {SEEDS}")
print(f"Total models trained: {len(SEEDS) * FOLDS * 2 * 2} (2 rounds)")
print(f"\nFeatures: {len(FEATURES)}")
print(f"Pseudo-labels: {len(test_pseudo)}")

print(f"\nFinal Results:")
print(f"  Round 1 Blend: {best_blend_auc:.5f}")
print(f"  Round 2 Blend: {best_blend_r2:.5f}")
print(f"  Improvement:   {best_blend_r2 - best_blend_auc:+.5f}")
print("="*80)